In [1]:

import copy
import xmltodict
import numpy as np
import json
import pprint

In [2]:
# % is the comment for SON

In [3]:
f = json.loads(open('m.json').read())

In [4]:
specs = f['specs']
ex = specs[-4]


#def read_schema()
xmltodict.parse(f['schema'][ex])['interleave'].keys()
d = xmltodict.parse(f['schema'][ex])

In [5]:
f['annotations'][':agents:KFacility']['vars']['in_capacity']

{'alias': 'in_capacity',
 'doc': 'number of commodity units that can be taken at each timestep (infinite capacity can be represented by a very large number)',
 'index': 3,
 'shape': [-1],
 'tooltip': 'input commodity capacity',
 'type': 'double',
 'uilabel': 'Incoming Throughput'}

In [6]:
archetypes = f['specs']
archetypes

[':agents:KFacility',
 ':agents:NullInst',
 ':agents:NullRegion',
 ':agents:Predator',
 ':agents:Prey',
 ':agents:Sink',
 ':agents:Source',
 ':cycamore:DeployInst',
 ':cycamore:Enrichment',
 ':cycamore:FuelFab',
 ':cycamore:GrowthRegion',
 ':cycamore:ManagerInst',
 ':cycamore:Mixer',
 ':cycamore:Reactor',
 ':cycamore:Separations',
 ':cycamore:Sink',
 ':cycamore:Source',
 ':cycamore:Storage']

In [7]:
conversion_dict = {'string': 'String',
                   'nonNegativeInteger': 'Int',
                   'boolean': 'Int',
                   'double': 'Real',
                   'positiveInteger': 'Int',
                   'float': 'Real',
                   'double': 'Real',
                   'duration': 'Int',
                   'integer': 'Int',
                   'nonPositiveInteger': 'Int',
                   'negativeInteger': 'Int',
                   'long': 'Real',
                   'int': 'Int',
                   'token': 'Real'
                   }

def read_element(eld, from_one_or_more=False, optional=False):
    if 'interleave' in eld.keys():
        s = read_interleave(eld['interleave'], eld['@name'], from_one_or_more, optional)
        return s

    # now there's optional and non-optional
    keys = eld.keys()
    if not from_one_or_more:
        options = {'MaxOccurs': 1}
    else:
        options = {}

    if optional:
        options = options
    else:
        options['MinOccurs'] = 1
        
    s = {eld['@name']: options}
    www = np.random.uniform(0, 10)
    if 'oneOrMore' in keys:
        s[eld['@name']].update(read_element(eld['oneOrMore']['element'],
                                       from_one_or_more=True, optional=optional)
                              )

        return s

    if 'data' in keys:
        options['ValType'] = conversion_dict[eld['data']['@type']]
        s[eld['@name']] = options
        return s

            
def read_interleave(intd, name, from_one_or_more, optional):
    if not optional:
        options = {'MinOccurs':1}
    else:
        options = {}
    if not from_one_or_more:
        options['MaxOccurs'] = 1
    d = {name: options}
    for i in intd['element']:
        d[name].update(read_element(i))
    return d



sep_dict = {}
d = xmltodict.parse(f['schema'][':cycamore:Separations'])
for i in json.loads(json.dumps(d['interleave']['element'])):
    sep_dict.update(read_element(i))
for j in json.loads(json.dumps(d['interleave']['optional'])):
    sep_dict.update(read_element(j['element'], optional=True))
pprint.pprint(sep_dict)

{'feed_commod_prefs': {'MaxOccurs': 1, 'val': {'ValType': 'Real'}},
 'feed_commods': {'MaxOccurs': 1,
                  'MinOccurs': 1,
                  'val': {'MinOccurs': 1, 'ValType': 'String'}},
 'feed_recipe': {'MaxOccurs': 1, 'ValType': 'String'},
 'feedbuf_size': {'MaxOccurs': 1, 'MinOccurs': 1, 'ValType': 'Real'},
 'latitude': {'MaxOccurs': 1, 'ValType': 'Real'},
 'leftover_commod': {'MaxOccurs': 1, 'ValType': 'String'},
 'leftoverbuf_size': {'MaxOccurs': 1, 'ValType': 'Real'},
 'longitude': {'MaxOccurs': 1, 'ValType': 'Real'},
 'streams': {'MaxOccurs': 1,
             'MinOccurs': 1,
             'item': {'MinOccurs': 1,
                      'commod': {'MaxOccurs': 1,
                                 'MinOccurs': 1,
                                 'ValType': 'String'},
                      'info': {'MaxOccurs': 1,
                               'MinOccurs': 1,
                               'buf_size': {'MaxOccurs': 1,
                                            'MinOccur

In [8]:
f['schema'][':agents:NullRegion']

'<text/>\n'

In [9]:
def check_if_list(d):
    # this is because some schemas,
    # if there's only one entry would have
    # a dictionary, while if there's multiple
    # would have a list
    if 'ict' in str(type(d)):
        return [d]
    else:
        return d
    

schema_dict = {}
template_dict = {}
for key in f['specs']:
    if 'NullRegion' in key or 'NullInst' in key:
        schema_dict[key] = {}
        continue
    libname = key.split(':')[1]
    schema_dict[key] = {}
    
    k = check_if_list(xmltodict.parse(f['schema'][key])['interleave']['element'])
    for i in k:
        schema_dict[key].update(read_element(i))
    k = check_if_list(xmltodict.parse(f['schema'][key])['interleave']['optional'])
    for i in k:
        schema_dict[key].update(read_element(i['element'], optional=True))

    template_dict[key] = schema_dict_string_to_template(schema_dict[key], key)


NameError: name 'schema_dict_string_to_template' is not defined

In [ ]:
pprint.pprint(schema_dict[key])

print(template_dict[key])

In [10]:
def reasonable_linebreak(string, lim=60):
        nlines = len(string) // lim

        space_indices = []
        for i in range(nlines):
            n = (i+1)*lim
            space_indices.append(string[n:].find(' ') + n)

        new_str = ''
        for indx, val in enumerate(string):
            if indx not in space_indices:
                new_str += val
            else:
                new_str += val + '\n'

        return new_str
    
    
def schema_dict_string_to_template(d, key):
    d = {key:d}
    c = copy.deepcopy(d)
    s = pprint.pformat(delete_keys_from_dict(c, ['MaxOccurs', 'MinOccurs', 'ValType']))
    s = s[1:]
    s = '\n'.join(s.split('\n'))
    s = s[:-1]
    s = s.replace("'", '')
    s = s.replace(',', '')
    s = s.replace(':', '')
    s = s.replace('"', '')
    s = s.split('\n')
    n = []
    for i in s:
        var = i.strip().split()[0]
        # print(dd[key])
        if var == 'streams':
            var = 'streams_'
        if var not in schema_dict[key].keys():
            # multiline variables with weird things
            print('skipping', var)
            continue
        # see if optional
       
        if 'MinOccur' not in d[key][var]:
            optional = '(optional)'
        else:
            optional = ''
        doc = reasonable_linebreak(optional + ' ' +f['annotations'][key]['doc'] ).split('\n')
        for j in doc:
            n.append('%' + j)
        n.append(i.strip())

    return '\n'.join(n)
    

def delete_keys_from_dict(dict_del, lst_keys):
    for k in lst_keys:
        try:
            del dict_del[k]
        except KeyError:
            pass
    for v in dict_del.values():
        if isinstance(v, dict):
            delete_keys_from_dict(v, lst_keys)

    return dict_del

    
def check_if_list(d):
    # this is because some schemas,
    # if there's only one entry would have
    # a dictionary, while if there's multiple
    # would have a list
    if 'ict' in str(type(element)):
        return [d]
    else:
        return d

In [3]:
from generate_sch import generate_schema
import pprint

In [4]:
obj = generate_schema('t')

In [5]:
from generate_sch import main
main()

In [6]:
! open ~/Downloads/Workbench-Darwin/bin/Workbench